In [153]:
import math
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

### Código para calcular la distancia geométrica entre un pixel y uno de sus vecinos.

In [166]:
def geoDist(x,y):
    
    (s,t) = x
    (m,n) = y
    
    numD = (s-m)**2 + (t-n)**2
    denD = 2*(10**2)
    
    return math.exp(((-1)*numD)/denD)

### Código para calcular la diferencia de intensidad entre un pixel y uno de sus vecinos.

In [155]:
def intDif(gx,gy):
    
    numI = (gx-gy)**2
    denI = 2*(6.7**2)
    
    return math.exp(((-1)*numI)/denI)

### Código para calcular la similitud entre un pixel y uno de sus vecinos.

In [156]:
def simil(img,x,y):
    
    (s,t) = x
    (m,n) = y
    
    gx = img[s,t]
    gy = img[m,n]
    
    return geoDist(x,y)*intDif(gx,gy)

### Código para calcular la similitud local de cada pixel en una matriz.

In [157]:
def imgSimil(img):
    
    imgS = img.copy()

    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            nx = []
            for p in posiciones:
                (i,j) = p
                if p!=(0,0):
                    if s+i >= 0 and s+i <= img.shape[0]-1 and t+j >= 0 and t+j <= img.shape[1]-1:
                        nx.append((s+i,t+j))
            dsetaX = 0
            for y in nx:
                dsetaX += simil(img,(s,t),y)
            imgS[s,t] = dsetaX
    
    return imgS

In [158]:
def imgLocalSimil(img, imgS):
    
    imgLS = img.copy()

    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            nx = []
            for p in posiciones:
                (i,j) = p
                if p!=(0,0):
                    if s+i >= 0 and s+i <= img.shape[0]-1 and t+j >= 0 and t+j <= img.shape[1]-1:
                        nx.append((s+i,t+j))
            dsetaY = 0
            for y in nx:
                (m,n) = y
                dsetaY += imgS[m,n]
            dsetaX = imgS[s,t]
            medDsetaX = dsetaX / (dsetaY/(len(nx)))
            if medDsetaX > 2.5:
                imgLS[s,t] = 1
            else:
                imgLS[s,t] = medDsetaX/2.5
    
    return imgLS

### Código para calcular la región designada de cada pixel.

In [159]:
def pixReg(img,imgLS):
    
    w1 = 2
    w2 = 4
    tSigma = 0.12
    
    flatReg = list()
    compReg = list()

    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            nx = []
            for p in posiciones:
                (i,j) = p
                if p!=(0,0):
                    if s+i >= 0 and s+i <= img.shape[0]-1 and t+j >= 0 and t+j <= img.shape[1]-1:
                        nx.append((s+i,t+j))
            a = 0
            b = 0
            for y in nx:
                (m,n) = y
                a += imgLS[m,n]**w1
                b += imgLS[m,n]**w2
            muX = 0
            for y in nx:
                (m,n) = y
                muX += ((imgLS[m,n]**w1)*img[m,n])/a
            sigmaX = 0
            for y in nx:
                (m,n) = y
                sigmaX += ((imgLS[m,n]**w2)*((img[m,n]-muX)**2))/b

            if sigmaX <= tSigma:
                flatReg.append((s,t))
            else:
                compReg.append((s,t))
    
    return (flatReg,compReg)

### Código para calcular la naturaleza de un pixel.

In [160]:
def pixNat(img,imgLS):
    
    noise = list()
    clean = list()

    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            nx = []
            for p in posiciones:
                (i,j) = p
                if p!=(0,0):
                    if s+i >= 0 and s+i <= img.shape[0]-1 and t+j >= 0 and t+j <= img.shape[1]-1:
                        nx.append((s+i,t+j))
            iX = img[s,t]
            yLS = list()
            yIntensidades = []
            for y in nx:
                (m,n) = y
                yLS.append(imgLS[m,n])
                yIntensidades.append((m,n))
            ind = yLS.index(max(yLS))
            (m,n) = yIntensidades[ind]
            iY = img[m,n]
            
            if abs(iX - iY) > 5:
                noise.append((s,t))
            else:
                clean.append((s,t))
    
    return (noise,clean)

### Código para calcular el nivel de ruido en la zona.

In [161]:
def noiseLvl(img,regiones,naturaleza):
    
    (flatReg,compReg) = regiones
    (noise,clean) = naturaleza
    
    sigma = 0
    acum = []
    
    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))

    if len(flatReg)==0:
        acum.append(0.60)
    elif len(flatReg)<10:
        for pix in range(10):
            (s,t) = flatReg[pix]
            nx = []
            for p in posiciones:
                (i,j) = p
                if p!=(0,0):
                    if s+i >= 0 and s+i <= img.shape[0]-1 and t+j >= 0 and t+j <= img.shape[1]-1:
                        nx.append((s+i,t+j))
            if flatReg[x] in noise:
                n = 1
                c = 0
            elif flatReg[x] in clean:
                n = 0
                c = 1
            for y in nx:
                if y in noise:
                    n += 1
                elif y in clean:
                    c += 1
            calculo = n / (c+n)
            acum.append(calculo)
    else:
        for pix in range(10):
            (s,t) = flatReg[pix]
            nx = []
            for p in posiciones:
                (i,j) = p
                if p!=(0,0):
                    if s+i >= 0 and s+i <= img.shape[0]-1 and t+j >= 0 and t+j <= img.shape[1]-1:
                        nx.append((s+i,t+j))
            if flatReg[x] in noise:
                n = 1
                c = 0
            elif flatReg[x] in clean:
                n = 0
                c = 1
            for y in nx:
                if y in noise:
                    n += 1
                elif y in clean:
                    c += 1
            calculo = n / (c+n)
            acum.append(calculo)
    sigma = sum(acum)/len(acum)
    
    print("Num. flat y comp reg:")
    print(len(flatReg))
    print(len(compReg))
    print(acum)
    print(sigma)

    thetaF = (-0.12)*(sigma**3) + 0.07*(sigma**2) + 0.75*sigma + 0.19
    thetaC = 0.31*(sigma**3) + 0.63*(sigma**2) + 0.52*sigma + 0.03
    
    
    return (thetaF,thetaC)

### Código para catalogar de forma final los píxeles de la imagen.

In [162]:
def filtrado(img,imgLS):
    regiones = pixReg(img,imgLS)
    naturaleza = pixNat(img,imgLS)
    (thetaF,thetaC) = noiseLvl(img,regiones,naturaleza)
    (flatR,compR) = regiones
    pixelesR = list()
    pixelesL = list()

    for p in flatR:
        (s,t) = p
        lsp = imgLS[s,t]
        if lsp <= thetaF:
            pixelesR.append(p)
        else:
            pixelesL.append(p)
    
    for p in compR:
        (s,t) = p
        lsp = imgLS[s,t]
        if lsp <= thetaC:
            pixelesR.append(p)
        else:
            pixelesL.append(p)
    
    medF = cv.medianBlur(img,5)
    gaussF = cv.GaussianBlur(medF,(5,5),0)

    for p in pixelesR:
        (s,t) = p
        if s == 0 or t == 0 or s == img.shape[0]-1 or t == img.shape[1]-1:
            valor = img[s,t] - gaussF[s,t]
            valorAb = abs(valor)
            if valorAb <= 15:
                pixelesL.append((s,t))
                pixelesR.remove((s,t))
    
    
    return (pixelesR,pixelesL)

### Código para calcular el valor de intensidad de un pixel ruidoso filtrado.

In [163]:
def valFinal(img,imgLS,ruidosos):
    
    resimg = img.copy()

    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    for r in ruidosos:
        (s,t) = r
        nx = []
        for p in posiciones:
            (i,j) = p
            if p!=(0,0):
                if s+i >= 0 and s+i <= img.shape[0]-1 and t+j >= 0 and t+j <= img.shape[1]-1:
                    nx.append((s+i,t+j))
        numG = 0
        denG = 0
        for y in nx:
            (m,n) = y
            lY = imgLS[m,n]
            if lY == 0:
                lY = 0.1
            numG += geoDist((s,t),y)*(lY**2)*img[m,n]
            denG += geoDist((s,t),y)*(lY**2)
        calculo = numG/denG
        resimg[s,t] = calculo
                
    return resimg

### Código de limpieza de imagenes con RVIN.

In [164]:
def limpiador(direccion):
    
    img = cv.imread(direccion, cv.IMREAD_GRAYSCALE)
    imgO = img.copy()
    imgS = imgSimil(img)
    imgLS = imgLocalSimil(img,imgS)
    (ruidosos,limpios) = filtrado(img,imgLS)
    
    PSNRp = 0
    PSNRim = 1
    acum = 0
    
    while PSNRim > PSNRp:
        acum += 1
        PSNRp = PSNRim
        imgR = valFinal(img,imgLS,ruidosos)
        PSNRim = cv.PSNR(imgO,imgR)
        imgO = imgR
        if acum > 9:
            PSNRp += PSNRim
    return imgO

In [165]:
imL = limpiador("images/noise60_lena_gray.png")
imO = cv.imread("images/noise60_lena_gray.png", cv.IMREAD_GRAYSCALE)
cv.imwrite("images/clean60_lena_gray.png",imL)
cv.imshow("Imagen con ruido",imO)
cv.imshow('Imagen arreglada',imL)
cv.waitKey(0)
cv.destroyAllWindows()

Num. flat y comp reg:
18657
243487
[0.7777777777777778, 0.9166666666666666, 0.8666666666666667, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.8666666666666667, 0.8666666666666667, 0.9333333333333333, 0.9333333333333333]
0.8961111111111112
